<h1>Basic test of the wflow BMI interface

In [ ]:
import wflow.wflow_bmi as bmi
import logging

reload(bmi)
%pylab inline 
import datetime
from IPython.html.widgets import interact

Startup two models:

+ The wflow_sbm model calculates the runoff from each cell (the LA land-atmosphere model)
+ the wflow_routing model that uses a kinimatic wave for routing the flow (the RT routing model)

In [ ]:
# This is the LAnd Atmophere (LA) model
LA_model = bmi.wflowbmi_csdms()
LA_model.initialize('../../examples/wflow_rhine_sbm/wflow_sbm.ini',loglevel=logging.ERROR)

# This is the routing (RT) model
RT_model = bmi.wflowbmi_csdms()
RT_model.initialize('../../examples/wflow_rhine_sbm/wflow_routing.ini',loglevel=logging.ERROR)

<h3>Now we can investigate some model parameters

In [ ]:
print(LA_model.get_value("timestepsecs"))


In [ ]:
imshow(LA_model.get_value("Altitude"))

In [ ]:
imshow(LA_model.get_value("Slope"))

In [ ]:
imshow(LA_model.get_value("FirstZoneDepth"))

In [ ]:
a =LA_model.get_value("TopoId")
a[a==-2147483648] = 0
imshow(a)

In [ ]:
imshow(LA_model.get_value("River"))

<h3>Start and end times

In [ ]:
t_end = RT_model.get_end_time()
t_start = RT_model.get_start_time()
t = RT_model.get_current_time()

(t_end  - t_start)/(86400)


<h3>Now start the models

In [ ]:
t_end = RT_model.get_end_time()
t = RT_model.get_start_time()

res = []
resq = []

# Loop in time and put output of SBM in seperate routing module - 1way link
while t < t_end:
    LA_model.update()
    # Now set the output from the LA model (specific Q) as input to the RT model
    thevar = LA_model.get_value("InwaterMM")
    RT_model.set_value("IW",thevar) # The IW is set in the wflow_routing.ini var as a forcing
    RT_model.update()
    resq.append(RT_model.get_value("SurfaceRunoff"))
    res.append(thevar)
    t = RT_model.get_current_time()
    
    print datetime.datetime.fromtimestamp(t)


In [ ]:
LA_model.finalize()
RT_model.finalize()

<h4>Define function to view the results

In [ ]:
def browse_res(digits):
    n = len(digits)
    def view_image(i):
        plt.imshow(log(digits[i]+1))
        plt.title('Step: %d' % i)
        plt.colorbar()
        plt.show()
    interact(view_image, i=(0,n-1))

In [ ]:
browse_res(res)


In [ ]:
 browse_res(resq)